# Hiblu: GPT LLM Fine-Tuning Query

<center><img src="https://imgtr.ee/images/2024/07/11/da26d77b15aa64e787c98038c1e6da32.png" alt="da26d77b15aa64e787c98038c1e6da32.png" border="0" /></center>

# Introduction

`fine-tuning` LLM model with pretrain GPT mode to create chatbot for Customer Service division to answer all question related to Blu digital banking services.

# Install Libraries

In [ ]:
!pip install openai==0.28
!pip install numpy
!pip install tiktoken

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Library

In [ ]:
import openai
import csv
import json
import os
import numpy as np
from collections import defaultdict
import tiktoken
import pandas as pd
from pathlib import Path

In [ ]:
openai.api_key = "Your key"

# Load Data

The data contains FAQs about Blu, consisting of 500 rows with 2 columns containing descriptions of questions and answers. This data will later be converted into JSON format and then trained on GPT.

In [ ]:
# Load CSV data in
csv_file_path = '/content/drive/MyDrive/finpro/FAQ_cleaned_final.csv'
df = pd.read_csv(csv_file_path)
df.head()

,Question,Answer
0,Apa itu blu?,blu merupakan aplikasi mobile banking dari BCA...
1,Apa perbedaan blu dengan BCA Digital?,blu adalah aplikasi mobile banking milik BCA D...
2,Apa perbedaan BCA Digital dengan BCA?,"BCA Digital merupakan anak perusahaan BCA, bag..."
3,Apa keuntungan pakai aplikasi blu?,"Gak terbatas ruang dan waktu, aplikasi blu bis..."
4,Apakah blu punya kantor cabang offline?,"blu gak punya kantor cabang offline, tapi tena..."


# Change To JSON

In [ ]:
# Define the system message
system_message = {"role": "system", "content": "Hi! HiBlu siap menjawab Pertanyaanmu!"}

# Initialize a list to hold all JSON objects
json_data = []

# Iterate over the rows of the dataframe
for index, row in df.iterrows():
    # Create the messages array for the current question-answer pair
    messages = [
        system_message,
        {"role": "user", "content": row['Question']},
        {"role": "assistant", "content": row['Answer']}
    ]
    # Append the JSON object to the list
    json_data.append({"messages": messages})

# Convert the list of JSON objects to a JSON string with each object on a new line
json_output = "\n".join(json.dumps(obj) for obj in json_data)

jsonl_file_path = '/content/drive/MyDrive/finpro/FAQ_cleaned-json.jsonl'
# Save the JSON output to a file
output_path = jsonl_file_path
with open(output_path, 'w',encoding='utf-8') as jsonl_file:
    jsonl_file.write(json_output)

print(f"JSON file saved to: {output_path}")

JSON file saved to: /content/drive/MyDrive/finpro/FAQ_cleaned-json.jsonl


In [ ]:
data_path = '/content/drive/MyDrive/finpro/FAQ_cleaned-json.jsonl'

# Load dataset
with open(data_path) as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 500
First example:
{'role': 'system', 'content': 'Hi! HiBlu siap menjawab pertanyaanmu!'}
{'role': 'user', 'content': 'Apa itu blu?'}
{'role': 'assistant', 'content': 'blu merupakan aplikasi mobile banking dari BCA Digital yang didesain dan dikembangkan untuk memberikan kemudahan untukmu agar bisa melakukan transaksi finansial dan non finansial melalui handphone.'}


# Check Json Format

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


# Calculate Token Price

In [ ]:
# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [ ]:
# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 55, 395
mean / median: 121.78, 105.0
p5 / p95: 71.0, 190.10000000000002

#### Distribution of num_assistant_tokens_per_example:
min / max: 12, 354
mean / median: 73.572, 55.5
p5 / p95: 24.900000000000006, 145.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~60890 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~182670 tokens


In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 3
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

In [ ]:
# Calculate the estimated cost for fine-tuning
cost_per_100k_tokens = 0.80  # Cost for every 100,000 tokens
estimated_cost = ((n_epochs * n_billing_tokens_in_dataset) / 100000) * cost_per_100k_tokens
print(f"Estimated cost for fine-tuning: approximately ${estimated_cost:.2f}") #I added this for actual cost based on current pricing

Estimated cost for fine-tuning: approximately $1.46


# Change Data Format

In [ ]:
# Function to save the dataset as a JSONL file
def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

# Specify the path where you want to save the JSONL file in your Google Drive
jsonl_file_path = '/content/drive/MyDrive/finpro/FAQ_cleaned-json-clean.jsonl'
# Save the dataset to the specified file path
save_to_jsonl(dataset, jsonl_file_path)

# Upload Data To GPT

In [ ]:
# Upload data for training
training_file_name = '/content/drive/MyDrive/finpro/FAQ_cleaned-json-clean.jsonl'

training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

# Gives training file id
print("Training file id:", training_file_id)

Training file id: file-at34w6BfH4AyQ895QwZ4Gq1d


# Create Fine Tuning "Job"

In [ ]:
# Create Fine-Tuning Job
suffix_name = "bluebca-bot"

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
    hyperparameters={
        "n_epochs": 3,  # Default: auto
        "batch_size": 32,  # Default: auto
        "learning_rate_multiplier": 0.1}  # Default: auto
)

job_id = response["id"]

print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-eqe41WYEtOIrIEL8B5IGC6NV",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1720088025,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-qye3BPz1mQfFglLyFLqIGIgJ",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-at34w6BfH4AyQ895QwZ4Gq1d",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 32,
    "learning_rate_multiplier": 0.1
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": "bluebca-bot",
  "seed": 154897632,
  "estimated_finish": null,
  "integrations": []
}


# Model Training

In [108]:
# Lisst events as fine-tuning progresses
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 2/47: training loss=1.90
Step 3/47: training loss=2.08
Step 4/47: training loss=2.03
Step 5/47: training loss=2.00
Step 6/47: training loss=2.01
Step 7/47: training loss=1.97
Step 8/47: training loss=1.82
Step 9/47: training loss=1.86
Step 10/47: training loss=1.94
Step 11/47: training loss=1.94
Step 12/47: training loss=1.84
Step 13/47: training loss=1.85
Step 14/47: training loss=1.80
Step 15/47: training loss=1.72
Step 16/47: training loss=1.60
Step 17/47: training loss=1.83
Step 18/47: training loss=1.78
Step 19/47: training loss=1.71
Step 20/47: training loss=1.68
Step 21/47: training loss=1.70
Step 22/47: training loss=1.60
Step 23/47: training loss=1.52
Step 24/47: training loss=1.75
Step 25/47: training loss=1.69
Step 26/47: training loss=1.73
Step 27/47: training loss=1.67
Step 28/47: training loss=1.78
Step 29/47: training loss=1.75
Step 30/47: training loss=1.63
Step 31/47: training loss=1.51
Step 32/47: training loss=1.72
Step 33/47: training loss=1.63
Step 34/47: trai

In [109]:
# Retrieve fine-tune model id
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-eqe41WYEtOIrIEL8B5IGC6NV",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1720088025,
  "finished_at": 1720088332,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:personal:bluebca-bot:9hDq9HEK",
  "organization_id": "org-qye3BPz1mQfFglLyFLqIGIgJ",
  "result_files": [
    "file-H0kIptpGBnzY0aX4jFj3Q9rr"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-at34w6BfH4AyQ895QwZ4Gq1d",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 32,
    "learning_rate_multiplier": 0.1
  },
  "trained_tokens": 179670,
  "error": {},
  "user_provided_suffix": "bluebca-bot",
  "seed": 154897632,
  "estimated_finish": null,
  "integrations": []
}

Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:bluebca-bot:9hDq9HEK


# Model Evaluation

In this phase, the model that has been fine-tuned will be tested for its credibility by presenting it with both FAQ-related questions and questions outside of the FAQ to see if the model can answer user-input questions accurately.

#### FAQ Related Questions

1. Apa itu blu?
2. Apa saja fitur blu?
3. Apa itu bluInvest?
4. Apa platform investasi yang bekerja sama dengan bluInvest?
5. Apa itu bluGether?
6. Apakah blu punya fitur pinjaman?
7. Bagaimana membuka rekening blu?
8. Bagaimana jika lupa password blu?
9. Apa itu bluGift?
10. Dimana saya bisa setor tunai tanpa kartu?

#### Non-related FAQ Questions

1. Siapa CEO blu?
2. Apa itu mobile legend?
3. Siapa itu Jokowi?
4. Kapan perang dunia 3?
5. Berapa harga laptop ASUS ROG Zephyrus G14 2023?
6. Apakah serial Game of Thrones endingnya bagus?
7. Siapakah artis hollywood dengan pembayaran tertinggi?
8. Bagaimana cara membuat kain batik?
9. Dimana saya bisa membeli piring cantik murah?
10. Berapa gaji data scientist?

## FAQ Related Questions

### Question 1

In [132]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "apa itu blu?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Blu adalah layanan perbankan digital yang disediakan oleh BCA. Dengan Blu, Anda dapat melakukan berbagai transaksi perbankan secara online, seperti transfer, pembayaran tagihan, dan pembelian pulsa.


### Question 2

In [133]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Apa saja fitur blu?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id,
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Fitur-fitur yang tersedia di blu antara lain transfer antar rekening BCA, transfer ke rekening bank lain, pembayaran tagihan, top up e-money, top up pulsa, dan fitur-fitur lainnya yang dapat memudahkan aktivitas perbankan Anda.


### Question 3

In [134]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Apa itu bluInvest?"
test_messages.append({"role": "user", "content": user_message})



# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

bluInvest adalah fitur investasi yang tersedia di aplikasi Blu BCA. Dengan bluInvest, Anda dapat melakukan investasi mulai dari Rp10.000 tanpa biaya tambahan.


### Question 4

In [135]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Apa platform investasi yang bekerja sama dengan bluInvest?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

BluInvest bekerja sama dengan platform Bareksa untuk layanan investasi.


### Question 5

In [137]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Apa itu bluGether?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

bluGether adalah fitur di aplikasi blu BCA yang memungkinkan pengguna untuk melakukan transaksi bersama-sama dengan teman atau keluarga melalui fitur "Bagi Tagihan" dan "Bagi Transfer".


### Question 6

In [139]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Apakah blu punya fitur pinjaman?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, Blu BCA tidak menyediakan fitur pinjaman. Blu BCA fokus pada layanan perbankan digital seperti transfer, pembayaran, dan top up. Apakah ada yang bisa saya bantu?


### Question 7

In [140]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Bagaimana membuka rekening blu?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id,
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Anda dapat membuka rekening Blu BCA melalui aplikasi Blu BCA yang dapat diunduh melalui App Store atau Google Play Store. Setelah mengunduh aplikasi, Anda dapat mengikuti langkah-langkah pendaftaran yang tertera di dalamnya.


### Question 8

In [141]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Bagaimana jika lupa password blu?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Jika Anda lupa password blu, Anda dapat melakukan reset password melalui fitur "Lupa Password" di aplikasi blu BCA. Anda akan diminta untuk memasukkan data-data yang diperlukan untuk verifikasi akun Anda.


### Questions 9

In [142]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Apa itu bluGift?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

bluGift adalah fitur di aplikasi blu BCA yang memungkinkan pengguna untuk memberikan uang elektronik kepada orang lain sebagai hadiah.


### Questions 10

In [143]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Dimana saya bisa setor tunai tanpa kartu?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Anda dapat melakukan setor tunai tanpa kartu di mesin setor tunai (CDM) yang tersedia di beberapa cabang BCA.


## Non-related FAQ Questions

### Question 11

In [144]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Siapa CEO blu?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

CEO blu adalah Pak Jahja Setiaatmadja.


### Question 12

In [145]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Apa itu mobile legend?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat memberikan informasi terkait dengan layanan blu BCA dan digital banking BCA. Apakah ada pertanyaan lain yang dapat saya bantu?


### Question 13

In [146]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Siapa itu Jokowi?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat membantu dengan pertanyaan terkait layanan Blu BCA dan digital banking BCA. Apakah ada yang bisa saya bantu terkait hal tersebut?


### Question 14

In [147]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Kapan perang dunia 3?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat membantu dengan pertanyaan terkait layanan Blu BCA atau layanan perbankan digital BCA. Apakah ada yang bisa saya bantu terkait hal tersebut?


### Question 15

In [148]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Berapa harga laptop ASUS ROG Zephyrus G14 2023?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat memberikan informasi terkait layanan blu BCA dan digital banking BCA. Apakah ada pertanyaan lain yang dapat saya bantu?


### Question 16

In [149]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Apakah serial Game of Thrones endingnya bagus?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat membantu dengan pertanyaan terkait blu BCA atau layanan perbankan digital BCA. Apakah ada yang bisa saya bantu terkait hal tersebut?


### Question 17

In [150]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Siapakah artis hollywood dengan pembayaran tertinggi?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat membantu dengan pertanyaan terkait blu BCA atau layanan perbankan digital BCA. Apakah ada yang bisa saya bantu terkait hal tersebut?


### Question 18

In [151]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Bagaimana cara membuat kain batik?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat membantu dengan pertanyaan terkait layanan blu BCA atau layanan perbankan digital BCA. Apakah ada yang bisa saya bantu terkait hal tersebut?


### Question 19

In [152]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Dimana saya bisa membeli piring cantik murah?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id,
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat membantu dengan pertanyaan terkait blu BCA atau layanan perbankan digital BCA. Apakah ada yang bisa saya bantu terkait hal tersebut?


### Question 20

In [153]:
# Test it out!
test_messages = []

system_message = "You work as a customer service representative at Blu, a digital banking service provided by BCA. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to blu BCA or BCA digital banking. And do not explain any application outside blu bca digital banking."
test_messages.append({"role": "system", "content": system_message})
user_message = "Berapa gaji data scientist?"
test_messages.append({"role": "user", "content": user_message})




# OpenAI Chat Completions
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, 
    messages=test_messages,
    temperature=0,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
print(response["choices"][0]["message"]["content"])

Maaf, saya hanya dapat memberikan informasi terkait dengan layanan blu BCA dan digital banking BCA. Apakah ada pertanyaan lain yang dapat saya bantu?


# Conclusion


Based on the provided evaluation:

* The model answered 8 out of 10 FAQ-related questions, but the answers were not exact replicas of the data; instead, the model generated more specific and concise sentences, unlike the RAG model which tends to provide descriptive overviews.
* In testing where questions were unrelated to the FAQ, the model correctly answered 9 out of 10 questions. However, the model attempted to answer a question related to a CEO, even though such data was not trained in the model.

Suggestions for this model development in the future:

* Using more detailed and representative prompts to influence the quality of the model's answers, as effective prompts can significantly impact the generated responses.
* Experimenting with other pretrained models like GPT-4 or GPT-40 to assess whether they can enhance the model's performance and accuracy.
* Trying out different temperature values to achieve optimal creativity in the model's responses.
* **Adding more training data** to enable the model to answer questions more accurately and broaden its knowledge base.

These approaches are aimed at improving the quality and versatility of the model in handling various types of questions and scenarios.